In [9]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


In [10]:
###
### Common ###
###

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()



In [11]:
from k12libs.utils.nb_easy import k12ai_get_top_dir, RACEURL
from k12libs.utils.nb_dataset import k12ai_generate_jsonfiles

In [12]:
data_root = '/raceai/data/datasets/cartoon_characters'
input_size = 224

In [13]:
info = k12ai_generate_jsonfiles(data_root)
info

{'num_records': 85,
 'num_classes': 5,
 'label_names': ['Monkey_D_Luffy',
  'sun_wukong',
  'Pleasant_Sheep',
  'Grey_Wolf',
  'Iron_man'],
 'mean': [0.6095, 0.6039, 0.589],
 'std': [0.2359, 0.2388, 0.2529]}

## Train

In [14]:
model = 'Resnet18'
data_source = f'{data_root}/train.json'
valid_data_source = f"{data_root}/val.json"
test_data_source = f"{data_root}/test.json"
root_dir = "/raceai/data/ckpts/cartoon_characters/"
resume_weights = f"{root_dir}/checkpoints/best.ckpt"

In [15]:
optimizer = {
    "class_name": "torch.optim.Adam",
    "params": {
        "lr": 0.001,
        "betas": [0.9, 0.999],
        "weight_decay": 0,
        "amsgrad": True
    }
}

scheduler = {
    "class_name": "torch.optim.lr_scheduler.ReduceLROnPlateau",
    "params": {
        "factor": 0.1,
        "patience": 3,
        "min_lr": 1e-6,
        "threshold": 1e-4,
        "verbose": True,
    }
}

early_stopping = {
    "class_name": "pytorch_lightning.callbacks.EarlyStopping",
    "params": {
        "monitor": "val_loss",
        "mode": "min",
        "patience": 6,
        "verbose": True,
        "min_delta": 0.0001
    }
}

model_checkpoint = {
    "class_name": "pytorch_lightning.callbacks.ModelCheckpoint",
    "params": {
        "monitor": "val_acc",
        "mode": "max",
        "save_top_k": 1,
        "period": 2,
        "save_weights_only": False,
        "filename": "best"
    }
}

logger = {
    "class_name": "pytorch_lightning.loggers.CSVLogger",
    "params": {
        "save_dir": root_dir,
        "name": "logs"
    }
}

random_rotation = {
    "class_name": "torchvision.transforms.RandomRotation",
    "params": {
        "degrees": 75
    }
}

reqdata = {
    "task": "cls.training.pl",
    "cfg": {
        "data": {
            "train": {
                "class_name": "raceai.data.process.LocalDataLoader",  
                "params": {
                    "data_source": data_source,
                    "dataset": {
                        "class_name": "raceai.data.JsonFileDataset",
                         "params": {
                             "input_size": input_size,
                             "mean": info['mean'],
                             "std": info['std'],
                             "imgaugs": [random_rotation]
                         }
                    },
                    "sample": {
                        "batch_size": 32,
                        "num_workers": 4,
                        "drop_last": True,
                        "shuffle": False
                    }
                }
            },
            "valid": {
                "class_name": "raceai.data.process.LocalDataLoader",  
                "params": {
                    "data_source": valid_data_source,
                    "dataset": {
                        "class_name": "raceai.data.JsonFileDataset",
                         "params": {
                             "input_size": input_size,
                             "mean": info['mean'],
                             "std": info['std']
                         }
                     },
                     "sample": {
                        "batch_size": 32,
                        "num_workers": 4,
                        "drop_last": False
                    }
                 }
            },
            "test": {
                "class_name": "raceai.data.process.LocalDataLoader",  
                "params": {
                    "data_source": test_data_source,
                    "dataset": {
                        "class_name": "raceai.data.JsonFileDataset",
                         "params": {
                             "input_size": input_size,
                             "mean": info['mean'],
                             "std": info['std']
                         }
                     },
                    "sample": {
                        "batch_size": 32,
                        "num_workers": 4,
                    }
                 }
            }
        },
        "model": {
            "class_name": f"raceai.models.backbone.{model}",  
            "params": {
                "device": 'gpu',
                "num_classes": info['num_classes'],
                "weights": True,
            }
        },
        "solver": {
            "optimizer": optimizer,
            "scheduler": scheduler,
            "trainer": {
                "class_name": "raceai.runner.pl.PlTrainer",
                "params": {
                    "callbacks": [model_checkpoint],
                    "logger": False,
                    "general": {
                        "max_epochs": 50,
                        "gpus": 1,
                        "resume_from_checkpoint": None,
                        "default_root_dir": root_dir
                    }
                }
            }
        }
    }
}

uri = f'{RACEURL}/raceai/framework/training'

resdata = json.loads(requests.post(url=uri, json=reqdata).text)
if 0 != resdata['errno']:
    if 'traceback' in resdata['result']:
        print(resdata['result']['traceback'])
    else:
        print(resdata)
else:
    print(resdata)

{'errno': 0, 'result': [{'test_acc': 0.8823529481887817}]}
